In [1]:
import SimpleITK as sitk
import pandas as pd
import os

In [2]:
# Swap information vector from PIR to RAS
def rearrange_vector( input_vector ) :
    output_vector = [input_vector[2], input_vector[0], input_vector[1]]
    return output_vector

In [3]:
input_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/converted_mouse_ccf/"
output_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/ccf_flipped_to_ras/"

data = { "input_file": [], "output_file": [] }

data["input_file"].append( os.path.join( input_directory, 'average_template','average_template_10.nii.gz') )
data["output_file"].append( os.path.join( output_directory,'average_template_10.nii.gz' ) )

data["input_file"].append( os.path.join( input_directory, 'annotation','ccf_2017','annotation_10.nii.gz') )
data["output_file"].append( os.path.join( output_directory,'annotation_10.nii.gz' ) )

df = pd.DataFrame( data )

In [4]:
for index, row in df.iterrows() :
    
    input_file = row['input_file']
    output_file = row['output_file']
    
    # -- Open input file
    print('========================')
    print('opening %s ...' % input_file )
    
    image = sitk.ReadImage( input_file )
    print("input size: " + str(image.GetSize()))
    print("input spacing: " + str(image.GetSpacing()))
    print("input direction: " + str(image.GetDirection()))
    
    #
    # -- Set coordinate origin to be the most dorsal (superior) tip
    # -- of the anterior commissure in the mid-sagittal plane
    # -- image index = (528, 489, 569.5)
    # 
    ac_index = (528,489,569.5)
    print("input ac index: " + str(ac_index))
    p = image.TransformContinuousIndexToPhysicalPoint(ac_index)
    neg_p = [-1.0 * round(x,3) for x in p]
    image.SetOrigin(neg_p)
    print("input origin: " + str(image.GetOrigin()))
    image.TransformPhysicalPointToContinuousIndex((0,0,0))
    
    
    #
    # -- Convert voxel organization
    # -- from PIR (+X=Posterior, +Y=Inferior(Ventral), +Z=Right) 
    # -- to RAS (+X=Right, +Y=Anterior, +Z=Superior)
    #
    reference_size  = rearrange_vector(image.GetSize())
    reference_spacing = rearrange_vector(image.GetSpacing())
    reference_direction = (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)  # RAS orientation
    reference = sitk.Image( reference_size, image.GetPixelIDValue() )
    reference.SetSpacing( reference_spacing )
    reference.SetDirection( reference_direction )   
    
    print("output size: " + str(reference.GetSize()))
    print("output spacing: " + str(reference.GetSpacing()))
    print("output direction: " + str(reference.GetDirection()))
    
    ac_index = rearrange_vector(ac_index)
    ac_index[1] = (reference_size[1]-1) - ac_index[1]
    ac_index[2] = (reference_size[2]-1) - ac_index[2]
    print("output ac index: " + str(ac_index))
    p = reference.TransformContinuousIndexToPhysicalPoint(ac_index)
    neg_p = [-1.0 * round(x,3) for x in p]
    reference.SetOrigin(neg_p)
    print("output origin: " + str(reference.GetOrigin()))
    reference.TransformPhysicalPointToContinuousIndex((0,0,0))
    
    #
    # -- resample output image
    #
    transform = sitk.ScaleTransform(3,(1.0,1.0,1.0))
    interpolator = sitk.sitkNearestNeighbor
    background_value = 0 
    
    print("resampling output image ...")
    resampled = sitk.Resample( image, reference, transform, interpolator, background_value )
    
    print("writing %s ..." % output_file )
    sitk.WriteImage( resampled, output_file, True )
    
    
    #
    # -- validate output
    #
    print( "validating output ...")
    roundTripped = sitk.Resample( resampled, image, transform, interpolator, background_value )
    
    inputArray = sitk.GetArrayFromImage( image )
    roundTrippedArray = sitk.GetArrayFromImage( roundTripped )
    comparison = (inputArray == roundTrippedArray )
    equal_arrays = comparison.all()
    
    if equal_arrays :
        print( "-- comparison PASSED" ) 
    else :
        print( "-- comparison FAILED" )
        

opening C:/Users/lydia/OneDrive/Work/DigitalAssets/converted_mouse_ccf/average_template\average_template_10.nii.gz ...
input size: (1320, 800, 1140)
input spacing: (0.009999999776482582, 0.009999999776482582, 0.009999999776482582)
input direction: (-0.0, 0.0, -1.0, 1.0, -0.0, 0.0, 0.0, -1.0, 0.0)
input ac index: (528, 489, 569.5)
input origin: (5.695, -5.28, 4.89)
output size: (1140, 1320, 800)
output spacing: (0.009999999776482582, 0.009999999776482582, 0.009999999776482582)
output direction: (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
output ac index: [569.5, 791, 310]
output origin: (5.695, 7.91, -3.1)
resampling output image ...
writing C:/Users/lydia/OneDrive/Work/DigitalAssets/ccf_flipped_to_ras/average_template_10.nii.gz ...
validating output ...
-- comparison PASSED
opening C:/Users/lydia/OneDrive/Work/DigitalAssets/converted_mouse_ccf/annotation\ccf_2017\annotation_10.nii.gz ...
input size: (1320, 800, 1140)
input spacing: (0.009999999776482582, 0.009999999776482582, 0.009